# <a id="index">INDEX</a>

[Setup](#0)

1. [Get Futures Data](#1)

2. [Get FX Data](#2)

3. [Get LEBAC DATA](#3)

4. [Merge and Save to Pickle](#4)

5. [Load Data](#5)

6. [Plot Data](#6)

# <a id="0">SETUP</a>
[Back to Index](#index)

In [1]:
#CODE SHOW TOGGLE
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Code Show."></form>''')

In [5]:
#INSTALL LIBRARIES
!yes | conda install qgrid
!echo "Done!"

Solving environment: done

# All requested packages already installed.

yes: standard output: Broken pipe
Done!


In [6]:
#IMPORT LIBRARIES

#% reset -f

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
from pandas.tseries.offsets import BMonthEnd
import datetime as dt
import seaborn as sns

import qgrid
from IPython.display import display
    
import numpy as np
import datetime

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets


# <a id="1">1. Get Futures Data</a>

[Back to Index](#index)

1.1 Go to http://www.rofex.com.ar/cem/Fyo.aspx 

1.2 Select all fields for DLR Dolar Com A 3500 except put and call ticks

1.3 Press view

1.4 Press "Exportar a Excel"

![title](screens/fig1.png)

In [0]:
!mv $HOME/Downloads/FyOROFEX.xls data/FyOROFEX.xls
!ls -l data/
df=pd.read_table("data/FyOROFEX.xls",decimal=",")
df.head()

total 1328936
-rw-r--r--@ 1 rcontesti  staff     580702 Dec 17 19:33 FyOROFEX.xls
-rw-r--r--  1 rcontesti  staff     395094 Dec 10 11:56 data.pkl
-rw-r--r--  1 rcontesti  staff     128552 Dec 10 17:56 lebac_sec
-rw-rw-r--@ 1 rcontesti  staff  679302606 Dec  8 07:39 series-tiempo.csv


,Fecha,Posicion,Tipo,Ejercicio,Primero,Minimo,Maximo,Ultimo,Volumen,Ajuste / Prima Ref.,Interes Abierto,PrimaDescuento,Unnamed: 12
0,15/12/2017 12:00:00 a.m.,DLR122017,Futuro,NaN,17.680,17.680,17.815,17.735,361750.0,17.735,1105801.0,NaN,NaN
1,15/12/2017 12:00:00 a.m.,DLR012018,Futuro,NaN,18.160,18.110,18.225,18.145,127939.0,18.145,735644.0,NaN,NaN
2,15/12/2017 12:00:00 a.m.,DLR022018,Futuro,NaN,18.492,18.470,18.580,18.500,107142.0,18.500,527157.0,NaN,NaN
3,15/12/2017 12:00:00 a.m.,DLR032018,Futuro,NaN,18.832,18.821,18.940,18.859,69107.0,18.854,328570.0,NaN,NaN
4,15/12/2017 12:00:00 a.m.,DLR042018,Futuro,NaN,19.315,19.200,19.320,19.244,39904.0,19.239,180449.0,NaN,NaN


In [0]:
## Format Data ##


df=pd.read_table("data/FyOROFEX.xls",decimal=",")


#Remplazar NA de precio por Ajuste

df["Ultimo"].fillna(df["Ajuste / Prima Ref."], inplace=True)


#Solo seleccionar los que necesito

futures=df[["Fecha","Posicion","Ultimo"]]


#Castear Fecha como Date en vez de DateTime

futures["Fecha"]=pd.to_datetime(df["Fecha"],dayfirst=True).apply(lambda x: x.date())



# Agregar Vencimiento

offset = BMonthEnd()

futures["venc_day"]=futures["Posicion"].apply(lambda x: 
                                              offset.rollforward(
                                                  dt.date(int(x[5:10]),int(x[3:5]),1)
                                              ).date()
                                             )


# Agregar dias al Vencimiento

futures["days_to_venc"]=(futures.venc_day-futures.Fecha).apply(lambda x: x.days)



futures.head()






/Users/rcontesti/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rcontesti/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rcontesti/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

,Fecha,Posicion,Ultimo,venc_day,days_to_venc
0,2017-12-15,DLR122017,17.735,2017-12-29,14
1,2017-12-15,DLR012018,18.145,2018-01-31,47
2,2017-12-15,DLR022018,18.500,2018-02-28,75
3,2017-12-15,DLR032018,18.859,2018-03-30,105
4,2017-12-15,DLR042018,19.244,2018-04-30,136


# <a id="1">2. Get FX Data</a>

[Back to Index](#index)



In [0]:
url="http://www.bcra.gov.ar/Pdfs/PublicacionesEstadisticas/com3500.xls"
fx=pd.read_excel(url,usecols="C,D",skiprows=[0,1,2],names=["Fecha","comA3500"])
fx["Fecha"]=fx["Fecha"].apply(lambda x: x.date())
fx.tail()

,Fecha,comA3500
3887,2017-12-11,17.2697
3888,2017-12-12,17.2635
3889,2017-12-13,17.3150
3890,2017-12-14,17.3567
3891,2017-12-15,17.5517


# <a id="3">3. Get LEBAC DATA</a>
[Back to Index](#index)

In [0]:
#Import dataset to df

col="dataset_titulo" 
dataset_titulo="Lebac mercado secundario"
url="http://infra.datos.gob.ar/catalog/sspm/dataset/304/distribution/\
304.1/download/lebac-mercado-secundario-tasa-interes-na.csv"

l2=pd.read_csv(url)

# Stack Dataset
l2=l2.melt(id_vars=["indice_tiempo"],var_name="var_name",value_name="tasa")
l2=l2[pd.isnull(l2.tasa)!=True]

#Caste Date
l2.indice_tiempo=pd.to_datetime(l2.indice_tiempo)

#Get Tipo de Tasa
l2["tipo"]=l2["var_name"].apply( lambda x: "_".join(x.split("_")[0:3]))

#Venciemiento
l2["venc"]=l2["var_name"].apply(lambda x: dt.date(int(x.split("_")[3]),int(x.split("_")[4]),int(x.split("_")[5])))
l2.venc=pd.to_datetime(l2.venc)

# Dias al Vencimiento
l2["venc_days"]=(l2.venc-l2.indice_tiempo).apply(lambda x: float(x.days))

# Precio Implicito de la Tasa
l2["precio"]=1000/(1+l2.tasa*l2.venc_days/365)

lebac=l2[["indice_tiempo","tipo","venc","venc_days","precio","tasa"]]
lebac.to_pickle("data/lebac_sec")
print(lebac.tail())
print("max date is {}".format(lebac.indice_tiempo.max()))
del l2, lebac

      indice_tiempo                 tipo       venc  venc_days     precio  \
12019    2017-11-27  lebac_interna_pesos 2018-06-21      206.0  856.83338   
12020    2017-11-28  lebac_interna_pesos 2018-06-21      205.0  857.37670   
12021    2017-11-29  lebac_interna_pesos 2018-06-21      204.0  857.95424   
12022    2017-11-30  lebac_interna_pesos 2018-06-21      203.0  858.61240   
12023    2017-12-01  lebac_interna_pesos 2018-06-21      202.0  859.22253   

           tasa  
12019  0.296054  
12020  0.296181  
12021  0.296229  
12022  0.296081  
12023  0.296053  
max date is 2017-12-01 00:00:00


# <a id="4">4. Merge and Save Data to pickle</a>

[Back to Index](#index)

In [0]:
#TODO: stack la tabla lebac con Posicion LEBMMYYY, o sea con las mismas columnas

data = pd.merge(futures, fx,  how='left', left_on='Fecha', right_on = 'Fecha')
data["rate"]=data.Ultimo/data.comA3500-1
data["ytm"]=((data.Ultimo/data.comA3500)**(365/data.days_to_venc))-1
data.to_pickle("data/data.pkl")
data.head()



,Fecha,Posicion,Ultimo,venc_day,days_to_venc,comA3500,rate,ytm
0,2017-12-15,DLR122017,17.735,2017-12-29,14,17.5517,0.010443,0.311096
1,2017-12-15,DLR012018,18.145,2018-01-31,47,17.5517,0.033803,0.294563
2,2017-12-15,DLR022018,18.500,2018-02-28,75,17.5517,0.054029,0.291861
3,2017-12-15,DLR032018,18.859,2018-03-30,105,17.5517,0.074483,0.283676
4,2017-12-15,DLR042018,19.244,2018-04-30,136,17.5517,0.096418,0.280233


# <a id="5">5. Load Data<a/>
[Back to Index](#index)

In [0]:
df=pd.read_pickle("data/data.pkl")

# Corregir el archivo Original Asi puedo borrar estas lineas
try:
    df["Fecha"]=df["Fecha"].apply(lambda x: x.date())
    df["venc_day"]=df["venc_day"].apply(lambda x: x.date())
except:
    pass



grid=qgrid.show_grid(df)
display(grid)


A Jupyter Widget

# <a id="6">6. Plot Data<a/>
[Back to Index](#index)

In [0]:

#Convert to Generic vars


def plot_tseries(filter,filter_value,dataframe,xvar,yvar,labels_var, y_in_perc, line=""):
    plt.close()
    fil=(dataframe[filter]==filter_value)
    ax=dataframe[fil].plot(x=xvar,y=yvar,marker="o",linestyle=line, figsize=(20,10), grid=1,legend=None,fontsize='17')

    # Set Ticks
    xtick = pd.date_range( dataframe[fil][xvar].min( ), dataframe[fil][xvar].max( ), freq='W' )
    ax.set_xticks( xtick, minor=True )
    #ax.grid('on', which='minor', axis='x' )
    #ax.grid('on', which='major', axis='x' )

    # Add Labels
    plt.xlabel(xvar.upper(),fontweight='bold', color = 'orange', fontsize='17', horizontalalignment='center')

    plt.ylabel(yvar.upper(),fontweight='bold', color = 'orange', fontsize='17', horizontalalignment='center')


    plt.gcf().autofmt_xdate()# Tilts dates so that they feet

    #Make Y Ticks percentages
    if y_in_perc==True:
        plt.axes().set_yticklabels(['{:3.2f}%'.format(l*100) for l in dataframe[fil][yvar]])# Percentage Format
    else:
        pass
    
    #Annotate
    I=dataframe[fil].index        
    if labels_var!="":
        for i in I:
            ax.annotate(dataframe[fil][labels_var][i],(dataframe[fil][xvar][i],dataframe[fil][yvar][i]),fontsize='17')
    else:
        i=dataframe[fil].index.min()
        ax.annotate(dataframe[fil][yvar][i],(dataframe[fil][xvar][i],dataframe[fil][yvar][i]),fontsize='17')

    
    plt.show()
    

interact(plot_tseries,
         filter=fixed("Fecha"),
         filter_value=widgets.Dropdown(
                                 options= list(df['Fecha'].unique()),
                                 value=datetime.date(2017, 12, 15),
                                 description='Fecha',
                                 disabled=False,
                                 layout=Layout(display='flex',
                                               flex_flow='row',
                                                align_items='stretch')                          
                                ),
         dataframe=fixed(df), # poniendo fixed lo quita de las opciones
         xvar="venc_day",
         yvar="ytm",
         labels_var="Ultimo",
         y_in_perc=fixed(True),
         line="--"
        )

A Jupyter Widget

<function __main__.plot_tseries>

In [0]:
interact(plot_tseries,
         filter=fixed("Posicion"),
         filter_value=widgets.Dropdown(
                                 options= list(df['Posicion'].unique()),
                                 value=df['Posicion'].unique().max(),
                                 description='Contrato',
                                 disabled=False,
                                ),
         dataframe=fixed(df), # poniendo fixed lo quita de las opciones
         xvar="Fecha",
         yvar="Ultimo",
         labels_var="",
         y_in_perc=fixed(False),
         line="--"
        )

A Jupyter Widget

<function __main__.plot_tseries>

In [0]:
# TODO: repetir los mismos graficos con PLOTLY y  DASH de PLOTLY Hacerce experto en estos. Los dos son open source y con licencia del MIT
